<h1>Basics of Mobile Robotics project<span class="tocSkip"></span></h1>

**07.12.2023**

**Group 23 : Diana Bejan** (325029)**, Emilie Grandjean** (286734)**, Garance Boesinger** (310447)**, Juan Martín** (376659)

**Table of contents**<a id='toc0_'></a>    
1. [Introduction](#toc1_)    
2. [General Setup](#toc2_)    
2.1. [Project description](#toc2_1_)    
2.2. [Environment Setup](#toc2_2_)    
2.3. [Best Path Calculations KIKE](#toc2_3_)    
2.4. [Motion Control](#toc2_4_)    
2.5. [Obstacles avoidance](#toc2_5_) 

3. [Required Components](#toc3_)   
3.1. [Computer Vision](#toc3_1_)    
3.2. [Global Navigation](#toc3_2_)    
3.3. [Motion Control](#toc3_3_)    
3.4. [Local Navigation](#toc3_4_)    
3.5. [Filtering](#toc3_5_)    

4. [VIDEOS ? IMAGES ? LITTLE EXTRAS ?](#toc4_) 

5. [Overall Project](#toc5_)    

6. [Conclusion](#toc6_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# 1. <a id='toc1_'></a>[Introduction](#toc0_)

As part of the Basics of Mobile Robotic class given by M. Mondada, we are asked to implement a project using the Thymio robot. This project involves the robot reaching a goal while moving in an environment filled with permanent obstacles. During it's path, some extra physical obstacles can be placed in its way. In such moments, the Thymio has to be able to avoid them and continue its route to the end point.

This is then done using several concepts seen in class such as image processing and pattern detection with Computer Vision, the Motion Control using a controller, Global and Local Navigation of the Thymio and Filtering.

In this report, we will talk about the different modules implemented and detail how they work to explain our overall functionnality of our project.

# 2. <a id='toc2_'></a>[General Setup](#toc0_)

## 2.1. <a id='toc2_1_'></a>[Project description](#toc0_)

1. **Create an environnment :** Our environment has to contain a set of obstacles that the Thymio avoids through *global navigation*. That is to say, the Thymio should avoid the obstacles without using the sensors to detect them.

2. **Find the best path :** The obective is that the Thymio goes from an *arbitrary point* in the map to a *target* that can be placed <u>anywhere</u> in the environment.  These will be changed during the demo to see how the system performs.

3. **Motion Control & Position estimation:** We will *control* the robot to help it move along the path. This requires an accurate estimate of the position of the robot which we will have to obtain through *bayesian filtering*.
   
4. **Avoid Obstacles :** While navigating, the Thymio will have to use *local navigation* to avoid *physical obstacals* that can be put in its path <u>at any point in time</u>. 

## 2.2. <a id='toc2_2_'></a>[Environment Setup](#toc0_)

Our environment setup has no boundary points and extend to the full viewing angle of the camera, so that the full picture detected can be exploited. It includes the **goal** (red square), **static obstacles** (black, shapes with sharp lines and edges, they cannot be curvilinear), a **background** (ideally white but it would work with any light color) and the **Thymio's position** (obtained with the colored circles, blue in the front and green in the middle, placed ontop of it). The choise of these features is highly related to the vision part as the image captured needs to be processed in the Computer Vision part of our project.

As for the static obstacles, we decided to have them in 2D to avoid any shadows that could compromise the detection by the camera. Further explanations on how these obstacles and features are detected is given in the Computer Vision part. 

An example of a frame captured by the camera with the different features mentionned is illustrated here below : 

![image.png](attachment:image.png)


## 2.3. <a id='toc2_3_'></a>[Best Path Calculations KIKE](#toc0_)

short explication on what method used and why and how and quickly mention that it uses the info send by the CV picked up by the environment

## 2.4. <a id='toc2_4_'></a>[Motion Control](#toc0_)

For the Motion Control, we decided to only set speeds. They are chosen with respect to the distance between the current position to the next point on the path as well as the differance between the current angle and the desired angle. These values are obtained through the Kálmán Filter and the Global Navigation. Essentially, the motion is a continuous exchange between turning and advancing phases.

## 2.5. <a id='toc2_5_'></a>[Obstacles avoidance](#toc0_)

For the obstacle avoidance section, we decided to use 3D physical obstacles that cannot be detected by the camera. This will push the use of the Thymio's sensors to detect these obstacles. Since they can be put at anytime anywhere in the robot's path, using **Local Navigation** will allow to avoid them and once the obstacle avoided, the robot need to go back to following it's given path.

# 3. <a id='toc3_'></a>[Required Components](#toc0_)

| Funtion | Input | Output |
|:------|:------|:------|
|   ` def fonction()`  | input input input | output output output

## 3.1. <a id='toc3_1_'></a>[Computer Vision](#toc0_)

<u>**Idea**</u>

The **Vision** module is mostly based on the OpenCV documentation. Other sources used are mentionned throughout the explanation. 

The purpose of the computer vision part, which is autonomous from the other parts but essential to compute the reference points of the setup, is to capture and use the picture from the camera to extract the main features of the environment, analyze them and give the required informations to the Global Navigation and Odometry modules (i.e. actual robot position corrdinates and angle, the goal and obstacles positions). The particularity of the Computer Vision module is that it does not communicate with the robot. 

The selected approach is based on color, shape and corners recognition. The first approach was with "perfect" images of a setup created virtually. Once switched to to the "noisy" pictures of the camera, we had to rescale quite a bit the values.

As for the code implemented, we created a vision class containing the basic arguments computed by vision and the functions that will be called in the main when needed. Some vision informations have to be recomputed at each step, such as the robot position (coordinates and angle), and some of them only need be computed once in the first step, such as the obstacles positions and their corners that do not move during execution. 

**1. Image capturing and image filtering**

At first, a frame is extracted from the camera. The camera has already been activated in the main (video registered in the variable **cap**). Note that we capture two frames because we noticed that sometimes, the initial connection of the camera to the computer, the first frame captured is "yellowish" and too bright to be used. Then, we filter the image several times to avoid noise and have a "smoother" picture to perform the next steps.

For the image filtering, we tested several combinations and the one in the function **capture_image()** seems to work best. Therefore, we perform 2D linear filtering, then blur the frame and then apply an additional median blur. Our filtered frame is stored in the vision Class so it can used by all the other vision functions. 

In [2]:
def capture_image(self,cap): 
    #capture a frame out of the video that will be used through CV part 
    ret, self.frame = cap.read()
    ret, self.frame = cap.read()
    kernel = np.ones((5,5),np.float32)/25
    img = cv2.filter2D(self.frame,-1,kernel)
    img = cv2.blur(img,(5,5))
    self.img = cv2.medianBlur(img,5)

**2. Goal position, thymio position and angle tracking**

To find the coordinates of the goal and start position, we use color filtering. We apply a color mask of the target color (red for the goal, blue and green for the robot) on the picture and then use maxLoc function or contour finding to extract the coordinates. The values of the color mask have been found using hsv color space graph and forums, they are empiric values adapted to our setup, and to the colors of the papers we are using. Indeed, one of the main "flaws" of the vision part, that will be discussed more in details in the weakness part, is that it is quite sensitive to light and color changes and uniformity.

The function find_goal_pos applies a red mask to the filtered frame, isolating the goal shape. Then we apply a contour detection on the thresholded image and find the moment, i.e. center of the shape. This value corresponds to the goal position (center of the red square). 

The functions find_start_pos and find_angle provide the coordinates of the two points on top of the thymio. As the circle is of small radius, we use maxLoc function to compute the coordinates, so it is not the exact center of the point but it is precise enough (more or less 10points on a grid of 480x640). The green point is used as starting point, i.e. as the position of the thymio. The blue one is used to compute the angle of the thymio. We obtain a positive angle going from zero to two py starting from the x axis and turning counterclockwise. We made several measures for the odometry part comparing measured value and real values, and the precision of the angle is +-0.01rad. 

At the end of the functions computing the positions and angle, we store those values in the shared Thymio Class to be used by the other functions of the code.

The *find_angle* function pasted below computes the coordinates of the blue point on the thymio at first in the same way the coordinates of the green (=start point) are computed. Then, using both green (back) and blue (front) coordinates, we compute the angle. 

Below is the thresholded image of the frame given as example with red mask applied, and of the green point on the robot, along with the coordinates computed by the functions. 

![2.png](attachment:2.png)

![3.png](attachment:3.png)

In [3]:
def find_angle(self,robot):
    # COORDINATES OF THE BLUE POINT 
    hsv = cv2.cvtColor(self.img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, self.LOW_BLUE, self.HIGH_BLUE)
    blue = cv2.bitwise_and(self.frame,self.frame, mask = mask)
    gray = cv2.cvtColor(blue, cv2.COLOR_RGB2GRAY) 
    _, self.thresh1 = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY) 
    (minVal, maxVal, minLoc, maxLoc) = cv2.minMaxLoc(self.thresh1)
    [x_front,y_front] = maxLoc
    # WE VERIFY ONE BLUE POINT IS ACTUALLY DETECTED TO AVOIR HIDING SITUATIONS (if the robot is hidden, we do not
    # recompute the coordinates)
    if (x_front != 0) or (y_front != 0):
        [self.x_front,self.y_front] = [x_front,y_front]
    # ANGLE COMPUTATION
        if self.y_front < self.y_back : 
            if self.x_front > self.x_back : 
                self.teta = np.arccos((self.x_front-self.x_back)/(np.sqrt(np.power((self.x_front-self.x_back),2)+np.power((self.y_front-self.y_back),2))))
            if self.x_front <= self.x_back : 
                self.teta = np.pi - np.arccos((self.x_back-self.x_front)/(np.sqrt(np.power((self.x_front-self.x_back),2)+np.power((self.y_front-self.y_back),2))))
        if self.y_front >= self.y_back : 
            if self.x_front > self.x_back : 
                self.teta = 2*np.pi - np.arccos((self.x_front-self.x_back)/(np.sqrt(np.power((self.x_front-self.x_back),2)+np.power((self.y_front-self.y_back),2))))
            if self.x_front <= self.x_back : 
                self.teta = np.pi + np.arccos((self.x_back-self.x_front)/(np.sqrt(np.power((self.x_front-self.x_back),2)+np.power((self.y_front-self.y_back),2))))
    # PASS THE VARIABLES TO THE SHARED THYMIO CLASS AND INDICATE THAT VISION IS DONE
        robot.setPositions(self.x_back,self.y_back,self.x_goal,self.y_goal,self.teta)
        robot.setVisionDone(True)
    # If hidden thymio, do not update the vision variables 
    else:
        robot.setVisionDone(False)

**3. Obstacles**

For the obstacles identification part, it depends on the type of global navigation used. As discussed in the next *Gloabl Navigation* section, we implemented two methods for global part, Astar and Visibility.  

In the case of visibility graph, the process used is the following:  
At first, we apply a big thresholding on the gray image to obtain a frame containing only the obstacles (for a noisy image, we set an empirical value of 40, which keeps only the black obstacles). The corners and moments of the obstacles are obtained using openCV corners and moments finding functions. We then implemented a small part of code placing the corners further away from the shapes to avoid the thymio running into the obstacles when following the side of a shape. The value of the distance of the corners to their shape is a variable that can be changed in the code, and that should typically be half the size of the thymio. We also took out the corners too close to the borders as we do not want the thymio to go outside the setup boundaries. Those methods are implememted in the function *find_corners*.  

Then we have to provide the *global navigation* part a "visibility graph" with the nodes and weights of each node to all others. Obviously the nodes considered are the corners plus start and goal position. We therefore compute a matrix containing the distance of each point to all others, implemented in the *compute_dist_mx* function. But we cannot just calculate the distance between points because if there is an obstacle in the way between two points, it must not be taken into account. We therefore decided to use threshold on the line to avoid this. Here is a quick explanation of the approach: on the thresholded image of the obstacles, if we extract the pixel values of the line between two points, if it contains black, there is an obstacle in the way and we do not compute the distance. 

To optimally use this 'line threshold' technique, and before calling compute_dist_mx function, we call the function *trace_contours* that traces the lines beteween the corners to avoid computing path in the "forbidden area" between obstacles and corners. 

Below is a picture of the frame used for this setup to compute the distance matrix, and an example of a distance matrix obtained for the setup provided as illustration. Distance_matrix should be read as follow: indices along vertical and horizontal dimensions are [start_point, corner1, corner2, ..., cornerN, goal_point], matrix_element[i,j] = distance between corner[i] and corner[j], and if there is an obstacle in the path, distance = 0. 

In [5]:
def find_corners(self):
    # Put the filtered image in grayscale
    gray = cv2.cvtColor(self.img, cv2.COLOR_BGR2GRAY) 
    # Threshold the image to keep only black elements
    ret, thresh2 = cv2.threshold(gray, self.BLACK_THRESHOLD, 255, cv2.THRESH_BINARY) 
    # Find contours of obstacles 
    contours, _ = cv2.findContours( 
        thresh2, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 
    self.thresh2 = thresh2
    i = 0
    m = []
    # Find moments, i.e. centers of obstacles 
    for contour in contours:  
        if i == 0: 
            i = 1
            continue
        M = cv2.moments(contour) 
        if M['m00'] != 0.0: 
            x = int(M['m10']/M['m00']) 
            y = int(M['m01']/M['m00']) 
            if len(m)<self.NB_SHAPES:
                m.append([x,y])
    th3 = cv2.adaptiveThreshold(self.thresh2,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
                                cv2.THRESH_BINARY,11,2)
    # Find corners of obstacles 
    corners = cv2.goodFeaturesToTrack(th3, self.NB_CORNERS, 0.01, 45) 
    corners = np.int0(corners) 
    # Part of code that changes the coordinates of the corners to put them away from the obstacles 
    
    ####*** CODE TO PUT THE CORNERS AWAY FROM THE OBSTACLES ***####

![5.png](attachment:5.png)

In [6]:
def trace_contours(self):
    a = int(np.size(self.cornerss)/2)
    b = int(np.size(self.m_cor)/2)
    for i in range (0,a):
        for j in range (0,b):
            if i != j:
                if self.m_cor[i] == self.m_cor[j]:
                    line = np.transpose(np.array(draw.line(self.cornerss[i][0], self.cornerss[i][1], self.cornerss[j][0], self.cornerss[j][1])))
                    data = self.thresh2[line[:, 1], line[:, 0]]
                    if np.size(np.where(abs(np.diff(data))>0)[0]) <= 2:
                        if np.mean(data) > self.mean_value_along_line:
                            cv2.line(self.thresh2, self.cornerss[i], self.cornerss[j], self.bluepx, 4)

![6.png](attachment:6.png)

In [7]:
def compute_dist_mx(self,robot):
    s = int(((np.size(self.cor))/2)+2) 
    dist_mx = np.zeros((s,s))
    for i in range(1,s):
        if i == (s-1):
            line = np.transpose(np.array(draw.line(self.x_back,self.y_back, self.x_goal,self.y_goal)))
            data = self.thresh2[line[:, 1], line[:, 0]]
            if np.size(np.where(abs(np.diff(data))>0)[0]) <= 3 : 
                dist_mx[i][0] = np.sqrt(np.power((self.x_goal-self.x_back),2)+np.power((self.y_goal-self.y_back),2))
            continue 
        line = np.transpose(np.array(draw.line(self.x_back,self.y_back,self.cor[i-1][0],self.cor[i-1][1])))
        data = self.thresh2[line[:, 1], line[:, 0]]
        if np.size(np.where(abs(np.diff(data))>0)[0]) <= 3 : 
            dist_mx[i][0] = np.sqrt(np.power((self.cor[i-1][0]-self.x_back),2)+np.power((self.cor[i-1][1]-self.y_back),2))
    for i in range(0,(s-2)):
        for j in range(0,s):
            if j == (i+1):
                continue 
            if j == 0 : 
                line = np.transpose(np.array(draw.line(self.x_back,self.y_back, self.cor[i][0],self.cor[i][1])))
                data = self.thresh2[line[:, 1], line[:, 0]]
                if np.size(np.where(abs(np.diff(data))>0)[0]) <= 3 : 
                    dist_mx[j][i+1] = np.sqrt(np.power((self.cor[i][0]-self.x_back),2)+np.power((self.cor[i][1]-self.y_back),2))
                continue 
            if j == int(s-1):
                line = np.transpose(np.array(draw.line(self.x_goal,self.y_goal, self.cor[i][0],self.cor[i][1])))
                data = self.thresh2[line[:, 1], line[:, 0]]
                if np.size(np.where(abs(np.diff(data))>0)[0]) <= 3 : 
                    dist_mx[j][i+1] = np.sqrt(np.power((self.cor[i][0]-self.x_goal),2)+np.power((self.cor[i][1]-self.y_goal),2))
                continue
            line = np.transpose(np.array(draw.line(self.cor[j-1][0],self.cor[j-1][1],self.cor[i][0],self.cor[i][1])))
            data = self.thresh2[line[:, 1], line[:, 0]]
            if np.size(np.where(abs(np.diff(data))>0)[0]) <= 3 : 
                dist_mx[j][i+1] = np.sqrt(np.power((self.cor[i][0]-self.cor[j-1][0]),2)+np.power((self.cor[i][1]-self.cor[j-1][1]),2))
    for i in range(0,(s-1)):
        if i == 0:
            line = np.transpose(np.array(draw.line(self.x_back,self.y_back, self.x_goal,self.y_goal)))
            data = self.thresh2[line[:, 1], line[:, 0]]
            if np.size(np.where(abs(np.diff(data))>0)[0]) <= 3 : 
                dist_mx[0][s-1] = np.sqrt(np.power((self.x_goal-self.x_back),2)+np.power((self.y_goal-self.y_back),2))
            continue 
        line = np.transpose(np.array(draw.line(self.x_goal,self.y_goal,self.cor[i-1][0],self.cor[i-1][1])))
        data = self.thresh2[line[:, 1], line[:, 0]]
        if np.size(np.where(abs(np.diff(data))>0)[0]) <= 3 : 
            dist_mx[i][s-1] = np.sqrt(np.power((self.cor[i-1][0]-self.x_goal),2)+np.power((self.cor[i-1][1]-self.y_goal),2))
    self.dist_mx = dist_mx
    robot.setDistMx(dist_mx)
    robot.setCor(self.cor)
    robot.setS(s)

Example of a distance matrix

Concerning the vision work for the Astar algorithm, we provide the global navigation part an occupancy matrix, containing 0 where the space is free and 1 where there is an obstacle. We first need to resize the image because Astar needs a smaller grid to operate quickly enough. We can set the resizing parameters in the code.  

We use the obstacle image and transform it into a matrix, and then apply a threshold on the values of the matrix to obtain a binary 0/1 matrix, and store the occupancy matrix in the shared robot Class instance. This is implemented in the function **return_occupancy_matrix()**. 

In [9]:
def return_occupancy_matrix(self,robot):
    dim = (self.width_resized, self.height_resized)
    img = cv2.resize(self.img, dim, interpolation = cv2.INTER_AREA)
    rszd = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    mtx = np.array(rszd)
    mx = (mtx < 20).astype(int)
    robot.occupancy_matrix = mx

**Discussion of the flaws of the computer vision part**

- The color filtering is fun and useful but with hsv space and masking, we explore a "range" of values (for example "reddish" values). In noisy pictures, if the range is too large, we will detect a little bit of this color somewhere it should not be, and on the contrary if the range is too small we will not detect the target, or we will detect it in one lightning condition and not the other. To have an optimal color detection, we should have "perfect" pictures which, in reality, do not exist. It is therefore an issue to be dealt with. In our case, once the picture was well filtered and the mask were adapted to the colors we used, it worked well in most situations. 

- The functions implementing the distance matrix, putting away corners and thresholding along lines have been computed fully "by hand" and are surely not optimized. Using existing libraries and tools, we could optimize those functions and clean them out.

- The visibility part is very sensitive to lightning conditions.

- Our setup and code would not work with faded colors, bright light spot somewhere on the frame, artificial colored light or unidentified obstacles.

- Our approach does not support round shaped obstacles.

- The algorithm needs to be provided the exact number of shapes and corners before the execution. Those variables can be easily modified in the code. 

**Implementation of Vision in the main**

At the beginning of the execution, a new instance of *vision Class* is created, along with a capture of the video taken by the camera. In the infinity loop, the vision functions are always called first as those values are needed for the rest of the program. It is implememted as follows:

- vision Class instance initialized outside the while loop at the beginning of the execution.

- initialization of the video captured from the camera.

In the infinity loop:  
- Every time, we capture a new frame, find the goal and thymio position and angle. 

- We only compute the contours and corners of the obstacles once.

- The distance matrix for the visibility graph (+ path recomputing discussed in the global navigation part) is computed the first time we execute the loop, plus everytime we are in a kidnapping situation. 

In [15]:
#Classes initialization
from classes import Thymio
from filtering import KalmanFilter
from vision import Vision
from global_visibility import Global_Nav

robot = Thymio() # Set Thym as class Thymio as initialization before the while
KF = KalmanFilter()
vision = Vision()
global_nav = Global_Nav()

#Video capturing 
cap = cv2.VideoCapture(1)

a = 0
while(1) :
    a = a + 1
    #VISION
    vision.capture_image(cap)
    vision.find_goal_pos()
    vision.find_start_pos(robot,a)
    vision.find_angle(robot)
    if (a == 1) : 
        vision.find_corners()
        vision.trace_contours()
    if ((robot.vision == 1) & (a == 1))or((a != 1) & (robot.kidnap == True) & (robot.vision == 1)):
        vision.compute_dist_mx(robot)
        global_nav.dijkstra(robot)
        global_nav.extract_path(robot)
        if robot.kidnap == True:
            robot.kidnap = False
    robot.vision = True
    #END VISION 
    
    ###***REST OF THE CODE***###

ModuleNotFoundError: No module named 'skimage'

**Sources :**

TP2 and 'Project Computer Vision tutorial' of the *Basics of mobile robotic* course  
OpenCV documentation: https://docs.opencv.org/4.x/  
https://www.geeksforgeeks.org/how-to-detect-shapes-in-images-in-python-using-opencv/  
https://www.geeksforgeeks.org/python-detect-corner-of-an-image-using-opencv/  
https://stackoverflow.com/questions/30331944/finding-red-color-in-image-using-python-opencv  
https://www.delftstack.com/fr/howto/python/opencv-inrange/   

## 3.2. <a id='toc3_2_'></a>[Global Navigation](#toc0_)

<u>**Idea**</u>

The **Global Navigation** module goal is to provide a path for the *motion control* module, based on the informations provided by the *vision* module. We implemented two different global navigation methods that both ended working, but do not use the same setup and variables. First, an explanation about visibility graph method will be given. We use the distance matrix between the points provided in the robot class instance and compute the best path using Dijkstra's algorithm in the function *dijkstra*, based on the pseudo-code we found here: https://www.baeldung.com/cs/dijkstra#:~:text=Dijkstra's%20Algorithm%20is%20a%20pathfinding,we%20reach%20the%20end%20node.  

The output of this function is a *graph* matrix containing, for each node, its state ('visited' or not), its score to the start point and, most importantly, its nearest node (to the start).


In [16]:
def dijkstra(self,robot):

    self.dist_mx = robot.getDistMx()
    self.s = robot.getS()
    graph = np.zeros((3,self.s))
    graph[0] = 3000 
    graph[0][0] = 0
    graph[2] = 42
    result = 0
    current_node = 0
    a = 0
    while 1:
        b = 0
        for i in range (0,self.s):
            if graph[1][i] == 0: 
                b = b + 1
                if graph[0][i] < graph[0][result]:
                    result = i
                if b == 1:
                    result = i
        current_node = result 
        graph[1][current_node] = 1
        for i in range (0,self.s):
            if self.dist_mx[i][current_node] != 0:
                if graph[1][i] == 0: 
                    new_score = graph[0][current_node] + self.dist_mx[i][current_node]
                    if new_score < graph[0][i]:
                        graph[0][i] = new_score 
                        graph[2][i] = current_node 
        if current_node == int(self.s-1):
            break
        for i in range (0,self.s):
            if graph[1][i] == 0: 
                if graph[0][i] < graph[0][result]:
                    result = i
                    if graph[0][result] == 3000:
                        break

    self.graph = graph 
    self.current_node = current_node 

We then use the function **extract_path()** to compute in a single array the coordinates of the path points, from start point (Thymio) to goal. This is done by extracting the coordinates using *graph* matrix, going from goal through each 'closest node' until start position. We then put this path in the shared robot class to be used by *motion control* module.  

Below is an example of a path computed by the Visibility Graph algorithm.  

In [17]:
def extract_path(self,robot):
    cor = robot.getCor()
    path = [self.current_node]
    node = self.current_node
    while node != 0:
        path.append(int(self.graph[2][node]))
        node = path[-1]
    path = np.flip(path)
    path_coord = [[robot.pos_X,robot.pos_Y]]
    a = 0
    for p in path : 
        a = a + 1 
        if a != 1:
            if a != np.size(path):
                i = int(p-1)
                path_coord.append(cor[i])
    path_coord.append([robot.goal_X,robot.goal_Y])
    robot.path = path_coord
    if len(path_coord)>1:
        robot.setAngle()
    self.path = path 
    self.path_coord = path_coord

![7.png](attachment:7.png)

Let us now discuss the *Global Navigation* algorithms for Astar method, using the occupancy_matrix provided by the *vision part*.  

BLABLA

**Discussion of the differences between Astar and Visibility**

- The Astar computes a more 'optimal' path than Visibility but is way slower for a big grid. The disadvantage is therefore that we have to resize the frame to a lower scale, loosing resolution, and therefore having to put only big square obstacles on defined emplacements.  

- The Visibility Graph forces us to navigate between points and most of the time to follow obstacles contours. But it is way faster than Astar, even on big grids, which allows us to use almost any shape and range of size for the obstacles.  

To use both methods, we therefore have to change the setup between testings.  

## 3.3. <a id='toc3_3_'></a>[Motion Control](#toc0_)

<u>**Idea**</u>

This module of our project is used to give the wheels the necessary speeds to make the right movements. Three functions are present : the PID controller **PID_control()**, the turning function **turn()**, and the forward motion function **go_to_next_point()**. We switch between two phases while going from one point to the next. First, the turning phase where we use **turn()** to get in the angle that allows the robot to only move forward to reach the next point. Second, the advancing phase, which advences to the next point. 

The module is only called when there is no obstacle detected by the robot's proximity sensors. The functions work in a point-to-point and moment-to-moment fashion. This means that the angles taken into account are the current angle given by the Kálmán filter as well as the goal angle calculated between the first and second points of the path. The speeds are calculated for the distance between the two aforementioned points rather than from the current position to the final goal position.



**1. PID Controller**

The code has been inspired by the PI controller implemented in the course MICRO-315 given by Prof. Mondada. The PID controller is used for angle correction and is an integral part of both the turning and the forward motion phases. 


| Funtion | Input | Output |
|:------|:------|:------|
|   ` def PIDController()`  | the error between the current angle and the desired one <br> the object of class Thymio that the changes apply to| the rotational speed to give to the wheels |


Indeed, we look at the actual angle, *current_angle*, given by the filter,  and the goal angle of the robot, *robot.goal_angle* and using the difference between the two, *error*, the integral error, *robot.int_error*, the previous error, *robot.prev_error*, as well as the time difference between two calls of the PID controller to calculate an optimal speed for attaining the desired angle. 

**Add some explenation about how get the time difference**

While in **turn()** it is the only calculation of speed, in **go_to_next_point()** it adds a rotational speed to a forward speed already calculated. The coefficients KP, KI and KD have been experimentally chosen during testing.



**2. Turning function**

The first function the program goes through between two points is the turning function, turn. This function takes as numerical inputs the current angle, given by the filter,  and the goal angle of the robot, as calculated from the path in the Thymio class. The PID controller is called for the calculation of the speed that is then passed to the robot by the functions setSpeedLeft/setSpeedRight found in the class Thymio. A helper variable, goal_reached_t is used to signal whether the turning phase has ended and the forward motion can start.



| Funtion | Input | Output |
|:------|:------|:------|
|   ` def turn()`  | the current angle <br> the object of class Thymio that the changes apply to  <br> the node which the information is sent to | none, the setting of the speeds is done internally |

**3. Forward motion function**

Once the turning phase is done, as signalled by *goal_reached_t*, the advancing phase can be accessed. This function calculates and sends speeds similar to the turning phase. The speed is proportionally controlled to slow down and smoothly reach the next point. The coefficient K has also been experimentally chosen during testing. We use the PID controller to add a rotational speed in order to correct the angle and so to make sure that the robot stays on track.

Once the goal is reached, the second point in the path is discarded (the first one in the current position as given by the camera) and a new goal angle is calculated using the **setAngle()** function found in the file *classes.py* . A helper variable goal_reached_f is used to signal that the motion phase has ended and a new turning motion can begin.

| Funtion | Input | Output |
|:------|:------|:------|
|   ` def go_to_next_point()`  | the current angle <br> the current_position <br> if an obstacle is detected <br> the node which the information is sent to | none, only internal changes and speed setting|

In [1]:
"Add code to show how this this works between two points"
from tdmclient import ClientAsync
import motion_control
from classes import Thymio

client = ClientAsync()
node = await client.wait_for_node()
await node.lock()
await node.wait_for_variables()

Thym=Thymio()
Thym.path=[[0,10],[2,50]]
Thym.pos_X=Thym.path[0][0]
Thym.pos_Y=Thym.path[0][1]
Thym.setAngle()

ConnectionRefusedError: [Errno 61] Connection refused

In [ ]:
print(Thym.goal_angle)

**4. Management of the reaching of the final goal position**

When the length of the path is down to one, the goal position has been reached and the work is done. We see this in the code for **go_to_next_point()**, where there is a condition that skips the function if the path length is too short. The speeds are set to 0 as a advancing phase has been completed, and no new angle is calculated nor is the path changed anymore, as only the current position is left. 

In [2]:
def go_to_next_point(current_angle, current_position, obstacle, robot, node): 
    if len(robot.path) > 1:
        '''code...'''
    else:   # Advancing phase in completed when close enough to the desired point
        robot.goal_reached_f = True
        robot.goal_reached_t = False
        robot.prev_error = 0
        robot.int_error = 0
        robot.prev_time = 0
        robot.setSpeedRight(0,node)
        robot.setSpeedLeft(0,node)
        robot.path.pop(1)   # Path shortened

    if len(robot.path) > 1:
        robot.setAngle()    # Angle changed with respect to the new goal
                
    else:
        pass    # Function skipped

As seen in the code above, the condition has to be introduced in the advancing phase completion as the* *robot.setAngle()** function takes the shortened path as an input before it is taken into account by the whole function in a future call.

## 3.4. <a id='toc3_4_'></a>[Local Navigation](#toc0_)

<u>**Idea**</u>

The **Local Navigation** module is inspired by the exercise session *Week 3 - Artificial Neural Networks* paragraph 5. The local_navigation.py file contains 3 functions, two verification functions and the actual function that makes the Thymio avoid obstacles which would be placed in its path.

**1. Testing the presence of an obstacle**

| Funtion | Input | Output |
|:------|:------|:------|
|   ` def test_saw_osb(Thymio, node, obs_threshold): `  | The class *Thymio* from the file *classes.py* to be able to modify the booleen variable **obs_avoided**, the threashold value to which the sensors detect the obstacle is close enough | returns *True* or *False* depending on if there's an obstacle or not, set the booleen variable **obs_avoided** to *False* if an obstacle is detected

The function **test_saw_obs()** is actually practically the same as *test_saw_wall* from the exercise but without the use of the *verbose* booleen and instead, when an obstacle is detected, the function sets a global booleen variable called **obs_avoided** to *false*. This variable is used in the Local obstacle avoidance function so that it stays in the loop until the obstacle is fully contoured.

This function take in input the obstacle threashold (*obs_threshold*) that is set in the 3rd function.

In [10]:
def test_saw_osb(Thymio, node, obs_threshold) :

    '''This function verrifies if one of the proximity horiontal sensors
        sees and obstacle within the giving threashold.'''
    
    if any([x>obs_threshold for x in node['prox.horizontal'][:-2]]):

        Thymio.obs_avoided = False  # Booleen to state when the obstacle has been avoided or not
        return True

    return False

**2. Testing in what direction to contourn**

| Function | Input | Output |
|:------|:------|:------|
|   `def clockwise(node):`  |  | returns *True* or *False* depending on if the obstacle is most on the right or the left of the robot and let the next function know weather to contourn clockwise or counterclockwise
  

Our obstacles are cylinders that the Thymio contourns. Therefore, we want to know whether to contourn it clockwise or counterclockwise. This depends on if the obstacle is more on the right or the left of the robot when it get's in its path. The function **clockwise()** tests which sensor between the sensor <span style='color: red;'>[1]</span> (left) and the sensor <span style='color: red;'>[3]</span> (right) is currently detecting the obstacle. If in fact the sensor on the left ([1]) has a higher value than the other one, the functionn returns *true*, otherwise it returns *false*. This function is only called in the Local obstacle avoidance function.

In [11]:
def clockwise(node) :

    '''This function verrifies if the obstacle to avoid is more on its right or left,
        therefore, the Thymio will contourn it accordingly.'''

    prox = list(node["prox.horizontal"]) + [0]
    if prox[1] > prox[3] :
        return True
    
    return False

**3. Local obstacle avoidance**

| Function| Input | Output |
|:------|:------|:------|
|   `def obstacle_avoidance(Thymio, node, client, motor_speed=100, obs_threshold=500):`  | The class *Thymio* from the file *classes.py* to be able to modify the booleen variable **obs_avoided**, the motor speed for the movement of the wheels, set to 100, the threashold value to which the sensors detect the obstacle set to 500 | no real output, but modification of **obs_avoided** booleen when the obstacle is avoided
  

The function **obstacle_avoidance()** can be devided in two states : the '*turning*' and the '*contourning*'.

As soon the function is called, it starts by setting two variables : **clockwise_true** to *false* and **prev_state** to "*turning*". Then, since the 1st function (**test_saw_osb()**) has been called, the booleen **obs_avoided** is *false* and so we enter a while loop. 

This is when we enter the state of '*turning*' where we test, thanks to the 2nd function (**clockwise()**) if the Thymio will have to contourn on the left or on the right of the obstacle. If the robot has to turn clockwise, the booleen **clockwise_true** is set to *true*, otherwise, it stays to *false*. The set '*turning*' makes the Thymio rotate on itself so its not facing the obstacle anymore. Once this is done, the state is set to '*contourning*'.

Still in the while loop, we enter the state of '*contourning*' where, depending on the value of te booleen **clockwise_true**, the Thymio will move in an arc circle around the obstacle and then back to the '*turning*' state, it rotates on itself again to be back facing in the right direction to allow going on with the Motion Control.

At the end of the '*contourning*' state, the booleen variable **obs_avoided** is set to *true*, meaning that the Thymio has finised avoiding the obstacle and that the Motion Control can pick up again.



In [ ]:
def obstacle_avoidance(Thymio, node, client, motor_speed=100, obs_threshold=500): #, clockwise = False):
    """
    Wall following behaviour of the FSM
    param motor_speed: the Thymio's motor speed
    param wall_threshold: threshold starting which it is considered that the sensor saw a wall
    param white_threshold: threshold starting which it is considered that the ground sensor saw white
    param verbose: whether to print status messages or not
    """

    clockwise_true = False  # Booleen to state if the Thymio has to contourn on the left or right
 
    prev_state = "turning" # Stated of movement of the Thymio
    
    while not Thymio.obs_avoided :     # As long as the obstacle isn't avoided, stay in the while loop 
    
        if test_saw_osb(Thymio, node, obs_threshold) :
            
            if prev_state == "turning": # little rotation on it's own to then do the contourning
  
                if clockwise(node) :
                    
                    Thymio.setSpeedLeft(motor_speed, node)
                    Thymio.setSpeedRight(-motor_speed, node)

                    clockwise_true = True   # Thymio needs to contourn the obstacle clockwise

                else :
                    # Thymio needs to contourn the obstacle counterclockwise
                    
                    Thymio.setSpeedLeft(-motor_speed, node)
                    Thymio.setSpeedRight(motor_speed, node)
                    
                prev_state = "contourning" # Change the state so the Thymio countourns the obstacle fully
        
        else:
            if prev_state == "contourning": 

                if clockwise_true :

                    Thymio.setSpeedLeft(motor_speed-40, node)
                    Thymio.setSpeedRight(motor_speed, node)

                    prev_state = "turning"

                    aw(client.sleep(18))

                    Thymio.setSpeedLeft(motor_speed, node)
                    Thymio.setSpeedRight(-motor_speed, node)

                    aw(client.sleep(2))

                    Thymio.obs_avoided = True  # obstacle has been avoided, change the state booleen
                    
                else :

                    Thymio.setSpeedLeft(motor_speed,node)
                    Thymio.setSpeedRight(motor_speed-40,node)

                    prev_state="turning"

                    aw(client.sleep(18))

                    Thymio.setSpeedLeft(-motor_speed,node)
                    Thymio.setSpeedRight(motor_speed,node)

                    aw(client.sleep(2))

                    Thymio.obs_avoided = True  # obstacle has been avoided, change the state booleen

        aw(client.sleep(0.1)) #otherwise, variables would not be updated

## 3.5. <a id='toc3_5_'></a>[Filtering](#toc0_)

<u>**Idea**</u>

The necessity of a filter comes from the fact that the Thymio is not a perfect robot. This means that the robot may not always act as expected. Therefore, we implement a filter which uses two sources of informations (the position of the Thymio given by the camera and the odometry based on internal measurements specific to the Thymio). The overall goal of the filter is to take into consideration possible imperfections of the robot and predict its actual real position. The **Filtering** module is inspired by the exercise session *Week 8 - Kalman Filter* paragraph 7.1. 


**Kalman Filter and sensors used**

<u>Filter</u>

The issue relies in the incertainty of the exact position of the robot and therefore the accuracy to perform the intended actions. We depend on two sources of information to obtain the position of the Thymio. Hence, from the many possible filters available, we decided to implement a classic Kalman Filter. 

This filter acts as an estimator for the position and the speed of the Thymio when provided with several source of information. Since the wheels don't have encoders, we use the measurements of the speed and the position provided by the Computer Vision. In addition, with the present noise being Gaussian noise, Kalman filters are a simpler representation of a distribution with the variables : mean and variance, and can be assumed these filters are Gaussian.

We also decided to use a classic Kalman Filter rather than a EKF (Extended Kalman Filter) because, even though our system is non linear, we linearize it before applying the Kalman. Therefore, using a classic Kalman is sufficient.

The **Filtering** module is inspired by the exercise session *Week 8 - Kalman Filter* paragraph 7.1. 





<u>Sensors</u>

Like mentioned above, since the Kalman Filter is Gaussian, it can be used to compare different sensor measurements : 

- Camera : from Computer Vision, the information measured is the Thymio's position - coordinates ($x$, $y$) and angle ($\theta$)
- Odometry : using the robot's speed sensors, the informations measured are the wheels speed ($\dot{x} = v_{x}$, $\dot{y} = v_{y}$) and the angular velocity ($\dot{\theta}$)

By using serveral source of measurements, we can allow the filter to function even if the camera is suddently unavailable and have our Motion Control still operate correctly.

We will see later on, but it is trough the **Odometry** that we linearize our system so that we can perform a classic KF rather than an EKF.


**Code**

<u>States</u>

The states we consider are the Thymio's position and its velocities. The best case senario is when we have the Computer Vison and the Odometry functionning. 

$$
States 
= 
%\begin{gather}
 \begin{bmatrix}
           x_{k} \\
           \dot{x_{k}}\\
           y_{k} \\
           \dot{y_{k}}\\
           \theta_{k} \\
           \dot{\theta_{k}}
 \end{bmatrix}
=
 \begin{bmatrix}
           x_{k} \\
           v_{x_{k}}\\
           y_{k} \\
           v_{y_{k}}\\
           \theta_{k} \\
           v_{\theta_{k}}
 \end{bmatrix}
%\end{gather}
$$

$$x_{t+1} = Ax_{t} + w_{t}$$
$$y_{t} = Hx_{t} + v_{t}$$

Where, $x_{k}$ is the state of the system at sample $k$, $\mathbf{A}$ is the matrix defining how the system evolves, $w_{k}$ is the state stochastic perturbation with covariance matrix $\mathbf{Q}$, $y_{k}$ is the measurement related to the state by the matrix $\mathbf{H}$ and $v_{k}$ is the measurement noise with covariance matrix $\mathbf{R}$.

The actual measurements $y_{k}$ depend on if we have Camera Vision as well as Odometry or not. On the other hand, we do not use values for known inputs $u_{k}$ and the impacting matrix $\mathbf{B}$ because we assume that the Thymio's speed is considered as a state and not an input. In other words, we do a sort of approximation of the real system to simplify the model. 

<u>Matrices</u>


- Therefore, we start by computing the sensors covariance matrix $\mathbf{R}$ :

$$
\mathbf{R}
=
\begin{bmatrix}
r_{11} & 0 & 0 & 0 & 0 & 0 \\
0 & r_{22} & 0 & 0 & 0 & 0 \\
0 & 0 & r_{33} & 0 & 0 & 0 \\
0 & 0 & 0 & r_{44} & 0 & 0 \\
0 & 0 & 0 & 0 & r_{55} & 0 \\
0 & 0 & 0 & 0 & 0 & r_{66} \\

\end{bmatrix}\\
$$

$
r_{11} = q_{cam,x} = 7.73 \quad
r_{22} = q_{\dot{x}} = 6.48\quad
r_{33} = q_{cam,y} = 7.73\quad
r_{44} = q_{\dot{y}} = 6.48\quad
r_{55} = q_{cam,\theta} = 0.0049\quad
r_{66} = q_{\dot{\theta}} = 0.615\quad
$


When there is no vision, the matrix looks like : 

$$
\mathbf{R}
=
\begin{bmatrix}
r_{22} & 0 & 0 \\
0 & r_{44} & 0 \\
0 & 0 & r_{66} \\

\end{bmatrix}\\
$$

with the same values for $r_{22}$, $r_{44}$ and $r_{66}$

The values for $r_{ii}$ are computed, from samples, as the standard deviation of the respective variables ($x_{k}$, $\dot{x_{k}}$, $y_{k}$, $\dot{y_{k}}$, $\theta_{k}$, $\dot{\theta_{k}}$).



- From the matrix $\mathbf{R}$, we compute the model covariance matrix $\mathbf{Q}$ :

$$
\mathbf{Q}
=
\begin{bmatrix}
q_{11} & 0 & 0 & 0 & 0 & 0 \\
0 & q_{22} & 0 & 0 & 0 & 0 \\
0 & 0 & q_{33} & 0 & 0 & 0 \\
0 & 0 & 0 & q_{44} & 0 & 0 \\
0 & 0 & 0 & 0 & q_{55} & 0 \\
0 & 0 & 0 & 0 & 0 & q_{66} \\

\end{bmatrix}\\
$$

$
q_{11} = q_{cam,x} = 7.73 \quad
q_{22} = q_{\dot{x}} = 1000\quad
q_{33} = q_{cam,y} = 7.73\quad
q_{44} = q_{\dot{y}} = 1000\quad
q_{55} = q_{cam,\theta} = 0.0049\quad
q_{66} = q_{\dot{\theta}} = 1000\quad
$

The values for $q_{11}$, $q_{33}$ and $q_{55}$ are the same as $r_{11}$, $r_{33}$ and $r_{44}$ because we suppose that the Camera Vision keeps the same error in the model as for the sensors. It gives us the possibility to figure out how wrong the position can become. On the other hand, the values $q_{22}$, $q_{44}$ and $q_{66}$ are set to "extremely" high values. This is because the uncertainty of the velocities are high and implies added noise. So, we want that the only way to update the velocities is from the **Odometry**. 

- We compute the matrix that relate the states to the measurements $\mathbf{H}$ :

$$

\mathbf{H_{w/Vision}}
=
\begin{bmatrix}
1 & 0 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 \\

\end{bmatrix}

\hspace{20 mm}

\mathbf{H_{w/outVision}}
=
\begin{bmatrix}
0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 \\

\end{bmatrix}\\

$$

Essentially, the best case senario would be to have the imputs from the two sources of sensors, the camera and the wheels' speed. This allows to have as much information available. In this case, the Kalman Filter can compare the two positions and have a more precise estimation. 

We decided to use as the second source the wheels speed instead of the accelerometer as internal measurements because we assumed it wouldn't be very precise values.

<u>Kalman Filter Function</u>


| Funtion | Input | Output |
|:------|:------|:------|
|   `def filter_kalman(self, X_est_pre, P_est_pre, Thymio) :`  | no function input but uses global variables from the file classes.py and the modified values from the functino **odometry_update** | no real output but updates values for the estimations of the states and the uncertainty, hence, the updated values for the Thymio's position

Therefore we have two situations to take into account when computing the Kalman Filter equations : when we have simultaneously Computer Vision and Odometry, and when we only have Odometry. 

In the function **filter_kalman**, an  <span style='color: red;'>if / else</span> condition is implemented to distinguish the 2 cases. When the Computer Vision is functionning, it set a booleen variable **vision** to *true*, so that the code enters in the first case, otherwise, it goes in the second case and uses the matrices set to be used in the situation of pure Odometry.


In [10]:
def filter_kalman(self, Thymio) :
     '''code...'''    
     if (Thymio.vision) : # If we have the Computer Vision and Odometry

          y = [[Thymio.pos_X],[self.v_X],[Thymio.pos_Y],[self.v_Y],[Thymio.theta],[self.v_Theta]] 

          H = np.eye(6)

          R = [[7.73, 0, 0, 0, 0, 0], 
               [0, 6.48, 0, 0, 0, 0],
               [0, 0, 7.73, 0, 0, 0],
               [0, 0, 0, 6.48, 0, 0],
               [0, 0, 0, 0, 0.0049, 0],
               [0, 0, 0, 0, 0, 0.615]]

          Thymio.vision = 0
     else : # if we only have the Odometry

          y = [[self.v_X],[self.v_Y],[self.v_Theta]]

          H = [[0, 1, 0, 0, 0, 0],
               [0, 0, 0, 1, 0, 0],
               [0, 0, 0, 0, 0, 1]]
          
          R = [[6.48, 0, 0],
               [0, 6.48, 0],
               [0, 0, 0.615]]
     '''...code'''



<u>Odometry</u>

The Odometry allows us to linearize the system so we're able to use the classic Kalman Filter rather than an EKF. Through the equations specified below, we go from a non-linear system with the Thymio wheeel speeds $v_{R}$ and $v_{L}$ to a linear system with cartesian speeds $v_{x}$ and $v_{y}$. The latter speeds are the ones used in the **filter_kalman** function to perform the estimations.

- Odometry equations

$$
    \begin{cases}
        v_{x} =\dot{x} = \frac{v_{R} + v_{L}}{2} * \cos\left (\theta + \left (\frac{\theta}{2}\right)\right) \\
        \\
        v_{y} = \dot{y} =\frac{v_{R} + v_{L}}{2} * \sin\left (\theta + \left (\frac{\theta}{2}\right)\right)\\ 
        \\
        v_{\theta} = \dot{\theta} = \frac{v_{R} - v_{L}}{d_{wheels}}
    \end{cases} 


\hspace{20 mm}


    \begin{cases}
        x_{update} = v_{x} * \Delta{t}\\
        \\
        y_{update} = v_{y} * \Delta{t}\\ 
        \\
        \theta_{update} = v_{\theta} * \Delta{t}
    \end{cases} 
$$

| Funtion | Input | Output |
|:------|:------|:------|
|   `def odometry_update(self, Thymio) :`  | no function input but uses global variables from the file **classes.py** | no real output but modifies values in the class of **KalmanFilter** for the **filter_kalman** function

In [11]:
def odometry_update(self, Thymio) :
        '''Odometry calculation'''

        self.speed_L = Thymio.motor_speed_left
        self.speed_R = Thymio.motor_speed_right
        
        self.wheel_dist = 9.5 # Distance between the wheel (where they touch the ground)

        delta_t = time.time() - pre_time            # time.time() to get the value of the time
        delta_S = (self.speed_R + self.speed_L) / 2                         # Forward speed
        self.v_Theta =  (self.speed_R - self.speed_L) / self.wheel_dist     # Angular velocity 

        # calculations of the variations of the speed of the Thymio
        self.v_X = delta_S * np.cos(Thymio.theta + self.v_Theta/2)          # SPEED in X
        self.v_Y = delta_S * np.sin(Thymio.theta + self.v_Theta/2)          # SPEED in y

        # update of the position (coordinates and angle/orientation) of the Thymio after time delta_t
        self.pos_X_odo = self.v_X * delta_t
        self.pos_Y_odo = self.v_Y * delta_t
        self.pos_theta_odo = self.v_Theta * delta_t

        # Update for the previous time with the current time for the next iteration
        pre_time = time.time()

**Summary**

In summary, the Kalman Filter seemed to be the best option and the fact that we linearize the system before, using the odometry, we do not need to use the Exteded version even with a non-linear system. 

We calculated the different parameters for our covariance matrices $\mathbf{R}$ and $\mathbf{Q}$ based on measured samples directly taken with the camera and the Thymio's movement.

**++impact of the filter on the program, precision etc (illustration avec et sans Kalman et quand camera est cachée)**

*The filter has a good impact on the system's precision and we can see that from the illustrations on the camera window. These illustrations come from a function that we deemed interesting to add. It draws ovals with diameters based on the covariance matrice's values in x and y.* 

*The filter is able to keep an estimation of the position of the robot and correct it, whether the camera is available or not. The prediction is of course more reliable when the camera sends the position of the robot to compute the prediction. We can see that from the oval growing as time passes by when the vision is obstructed.*

*Finally, we would say that applying a filter to a small and not so complex system such as the Thymio robot, in an application lasting a few seconds does not have such a impactful influence. However we are now able to understand how important and meaningful to have one on an autonomous system such as a car, a satellite, etc. (basically all linears system with trajectory estimations). During a long-term mission, we cannot expect to cancel the error accumulation, but we can lessen it, hence the usefulness of Bayes filter.*

# 4. <a id='toc4_'></a>[VIDEOS ? IMAGES ? LITTLE EXTRAS ?](#toc0_)

# 5. <a id='toc5_'></a>[Overall Project](#toc0_)

**do we have to put the main here ? or a code that called our main.py ? to be asked on tuesday maybe**

# 6. <a id='toc6_'></a>[Conclusion](#toc0_)

**we need to write a conclusion**

some difficulties we have faced are entre autre ...